<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7f64f09dfdd976816725d1e372c70b37c93b992105cf7c99e7e5c0b56c762deb
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-18 09:55:15
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.38 C
-------------------
Today PnL: -1.06 L (-0.76%)
Current PnL: -28.81 L (-18.61%)
CY Booked + Current PnL: -14.52 L (-9.38%)
-------------------
Total profit:  1.22 L
Total loss:  -30.02 L
-------------------
Total Booked + Current PnL: 12.46 L (9.75%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.34%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 93.44 L (67.59%)
Deployed:  1.28 C
Current:  1.38 C
CAGR/XIRR %: 4.34%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-1.46,-20.34,25.65,0.09,20591.0,-20500.0,80277.0,86.34,26.0,M-SC,2.38,253.0,-1.00,0.58,5.47,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.63,8.68,10.14,19.69,21130.0,16638.0,208380.0,-1.91,56.0,X-MC,2.50,78.0,0.79,1.51,16.06,XY25,NTT,AC
50,MASFIN,398.61,-1.17,-4.67,28.03,22.05,26181.0,-4575.0,93405.0,-17.80,47.0,H-SC,6.75,164.0,-0.17,0.68,35.77,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.89,-16.49,19.77,0.02,28820.0,-28785.0,145774.0,122.10,41.0,M-SC,13.65,234.0,-1.00,1.06,26.80,OX40N,NTT,PAINTS
43,ITC,452.00,0.34,-1.68,12.68,10.78,29960.0,-4041.0,236277.0,-41.56,45.0,X-LC,2.05,5.0,-0.13,1.72,3.47,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,ROUTE,2227.21,1.76,-47.44,221.62,69.05,155005.0,-63124.0,69942.0,-57.43,51.0,H-SC,22.54,140.0,-0.41,0.51,7.87,SR,ATH,IT
15,CAMS,950.00,1.21,-2.69,27.91,24.48,68405.0,-6765.0,245091.0,-81.54,41.0,X-SC,0.60,86.0,-0.10,1.78,19.83,X40N,NTT,MISC
41,INFY,1972.00,1.03,10.18,21.84,34.24,75291.0,31852.0,344740.0,-14.21,65.0,X-LC,4.55,2.0,0.42,2.51,19.31,X40,NTT,IT
84,WIPRO,420.00,0.86,8.03,59.46,72.27,108845.0,13608.0,183056.0,-7.44,68.0,M-LC,2.80,99.0,0.13,1.33,14.70,XR,NTT,IT
13,BSOFT,831.70,0.80,-21.08,91.06,50.79,101078.0,-29651.0,111002.0,1.28,66.0,H-SC,6.62,151.0,-0.29,0.81,29.24,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.0,-3.03,1.50,18.98,20.77,45106.0,3508.0,237652.0,-7.31,38.0,X-MC,2.31,62.0,0.08,1.73,8.52,X40N,NTT,BREWERIES
56,RAJESHEXPO,518.0,-3.00,-63.33,172.90,0.07,86969.0,-86877.0,50300.0,1743.43,51.0,L-SC,2.36,268.0,-1.00,0.37,25.10,OX40N,NTT,JEWELLERY
27,GLAXO,3466.2,-2.55,1.03,37.73,39.15,75964.0,2052.0,201336.0,-18.92,43.0,X-MC,10.57,60.0,0.03,1.46,30.33,X40,ATH,PHARMA
44,JCHAC,2282.0,-2.45,-36.79,58.20,-0.01,47014.0,-47025.0,80780.0,16421.74,32.0,M-SC,8.93,233.0,-1.00,0.59,3.39,OX40N,NTT,AC
80,VALIANTORG,838.0,-2.42,-46.00,222.49,74.15,196568.0,-75256.0,88349.0,-259.08,40.0,H-SC,22.85,149.0,-0.38,0.64,13.25,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.57,-3.51,115.47,107.91,164505.0,-5178.0,142466.0,-21.36,51.0,M-SC,12.50,216.0,-0.03,1.04,6.23,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.89,-16.49,19.77,0.02,28820.0,-28785.0,145774.0,122.10,41.0,M-SC,13.65,234.0,-1.00,1.06,26.80,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.46,-20.34,25.65,0.09,20591.0,-20500.0,80277.0,86.34,26.0,M-SC,2.38,253.0,-1.00,0.58,5.47,OX40N,NTT,DURABLES
66,SIS,528.00,-0.14,-24.31,60.22,21.26,50606.0,-26997.0,84035.0,1982.38,49.0,H-SC,4.97,166.0,-0.53,0.61,13.54,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.81,-26.49,54.36,13.47,107765.0,-71424.0,198243.0,-69.87,30.0,H-SC,6.08,158.0,-0.66,1.44,0.80,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.86,8.03,59.46,72.27,108845.0,13608.0,183056.0,-7.44,68.0,M-LC,2.8,99.0,0.13,1.33,14.7,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.86,8.03,59.46,72.27,108845.0,13608.0,183056.0,-7.44,68.0,M-LC,2.80,99.0,0.13,1.33,14.70,XR,NTT,IT
25,FINCABLES,1641.55,-0.57,-3.51,115.47,107.91,164505.0,-5178.0,142466.0,-21.36,51.0,M-SC,12.50,216.0,-0.03,1.04,6.23,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.00,-3.87,115.04,106.72,136398.0,-4770.0,118566.0,-52.34,37.0,H-SC,3.44,139.0,-0.03,0.86,18.61,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.45,-3.62,40.72,35.63,82252.0,-7580.0,201994.0,-17.18,36.0,H-MC,2.77,119.0,-0.09,1.47,12.45,AR,ATH,PHARMA
37,IEX,219.00,-1.08,-5.87,57.89,48.63,108958.0,-11738.0,188216.0,-37.43,43.0,H-SC,13.56,136.0,-0.11,1.37,5.00,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.62,-19.48,93.09,55.47,182360.0,-47390.0,195896.0,-28.68,22.0,X-MC,13.97,64.0,-0.26,1.42,0.37,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.93,-11.33,44.47,28.11,79423.0,-22810.0,178600.0,-35.73,24.0,X-MC,13.48,55.0,-0.29,1.30,0.00,X40,ATH,APPARELS
8,AWL,485.00,-1.53,-24.76,104.00,53.50,236133.0,-74700.0,227051.0,-63.63,26.0,X-MC,3.80,58.0,-0.32,1.65,0.85,XY24,NTT,FMCG
75,TMPV,600.00,-2.06,-30.48,76.89,22.98,120756.0,-68848.0,157050.0,-27.96,26.0,X-LC,6.77,3.0,-0.57,1.14,0.00,XY24,NTT,AUTO
2,ABBOTINDIA,35195.00,-0.45,-7.73,27.96,18.07,46142.0,-13825.0,165030.0,-21.05,28.0,X-MC,6.34,63.0,-0.30,1.20,9.36,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,1.21,-2.69,27.91,24.48,68405.0,-6765.0,245091.0,-81.54,41.0,X-SC,0.60,86.0,-0.10,1.78,19.83,X40N,NTT,MISC
12,BERGEPAINT,680.0,-1.20,-5.15,27.71,21.14,59754.0,-11697.0,215642.0,-11.79,36.0,X-MC,1.68,74.0,-0.20,1.57,21.55,XY24,NTT,PAINTS
20,DABUR,735.0,-0.64,-4.00,49.69,43.70,118329.0,-9933.0,238135.0,-11.99,33.0,X-MC,1.98,72.0,-0.08,1.73,10.44,XY24,BTT,FMCG
43,ITC,452.0,0.34,-1.68,12.68,10.78,29960.0,-4041.0,236277.0,-41.56,45.0,X-LC,2.05,5.0,-0.13,1.72,3.47,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.62,3.69,26.08,30.73,53274.0,7267.0,204272.0,-20.57,51.0,X-MC,2.27,75.0,0.14,1.48,16.93,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.39,-26.37,122.93,64.15,215386.0,-62741.0,175210.0,-56.75,30.0,X-SC,3.02,82.0,-0.29,1.27,0.0,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.76,-39.87,124.86,35.20,96602.0,-51302.0,77368.0,1.37,28.0,X-SC,16.48,92.0,-0.53,0.56,0.0,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-0.74,-21.33,79.87,41.51,165491.0,-56165.0,207200.0,-6.31,32.0,X-MC,7.54,61.0,-0.34,1.51,0.0,XY25,ATH,FMCG
55,QUESS,424.00,-1.99,-32.31,111.02,42.84,48853.0,-21002.0,44004.0,-55.92,35.0,X-SC,35.23,83.0,-0.43,0.32,0.0,XY24,NTT,MISC
59,RELAXO,1176.00,-1.43,-48.78,197.35,52.31,146738.0,-70806.0,74354.0,-45.50,36.0,X-SC,7.30,91.0,-0.48,0.54,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.99,-32.31,111.02,42.84,48853.0,-21002.0,44004.0,-55.92,35.0,X-SC,35.23,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-1.43,-48.78,197.35,52.31,146738.0,-70806.0,74354.0,-45.50,36.0,X-SC,7.30,91.0,-0.48,0.54,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.76,-39.87,124.86,35.20,96602.0,-51302.0,77368.0,1.37,28.0,X-SC,16.48,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.90,-6.40,32.56,24.08,38690.0,-8124.0,118826.0,-13.99,29.0,X-SC,5.44,89.0,-0.21,0.86,11.96,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.83,-17.58,73.89,43.32,99190.0,-28632.0,134240.0,-29.17,33.0,X-SC,5.03,90.0,-0.29,0.98,3.07,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.34,-11.22,35.97,20.71,120782.0,-42452.0,335785.0,-23.62,63.0,X-LC,4.05,1.0,-0.35,2.44,12.20,X40,ATH,IT
41,INFY,1972.00,1.03,10.18,21.84,34.24,75291.0,31852.0,344740.0,-14.21,65.0,X-LC,4.55,2.0,0.42,2.51,19.31,X40,NTT,IT
75,TMPV,600.00,-2.06,-30.48,76.89,22.98,120756.0,-68848.0,157050.0,-27.96,26.0,X-LC,6.77,3.0,-0.57,1.14,0.00,XY24,NTT,AUTO
81,VBL,671.64,-1.82,-6.62,45.28,35.67,133552.0,-20895.0,294947.0,-17.64,43.0,X-LC,3.89,4.0,-0.16,2.14,6.32,X40N,ATH,FMCG
43,ITC,452.00,0.34,-1.68,12.68,10.78,29960.0,-4041.0,236277.0,-41.56,45.0,X-LC,2.05,5.0,-0.13,1.72,3.47,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.16,-14.48,112.44,81.67,90206.0,-13584.0,80226.0,7211.11,53.0,L-SC,12.36,271.0,-0.15,0.58,55.36,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.17,-4.67,28.03,22.05,26181.0,-4575.0,93405.0,-17.80,47.0,H-SC,6.75,164.0,-0.17,0.68,35.77,XR,ATH,FINANCE
13,BSOFT,831.70,0.80,-21.08,91.06,50.79,101078.0,-29651.0,111002.0,1.28,66.0,H-SC,6.62,151.0,-0.29,0.81,29.24,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.89,-16.49,19.77,0.02,28820.0,-28785.0,145774.0,122.10,41.0,M-SC,13.65,234.0,-1.00,1.06,26.80,OX40N,NTT,PAINTS
84,WIPRO,420.00,0.86,8.03,59.46,72.27,108845.0,13608.0,183056.0,-7.44,68.0,M-LC,2.80,99.0,0.13,1.33,14.70,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.80,-21.08,91.06,50.79,101078.0,-29651.0,111002.0,1.28,66.0,H-SC,6.62,151.0,-0.29,0.81,29.24,XR,ATH,IT
84,WIPRO,420.00,0.86,8.03,59.46,72.27,108845.0,13608.0,183056.0,-7.44,68.0,M-LC,2.80,99.0,0.13,1.33,14.70,XR,NTT,IT
41,INFY,1972.00,1.03,10.18,21.84,34.24,75291.0,31852.0,344740.0,-14.21,65.0,X-LC,4.55,2.0,0.42,2.51,19.31,X40,NTT,IT
73,TCS,4389.97,0.34,-11.22,35.97,20.71,120782.0,-42452.0,335785.0,-23.62,63.0,X-LC,4.05,1.0,-0.35,2.44,12.20,X40,ATH,IT
32,HCLTECH,1922.01,0.22,7.64,15.87,24.73,41328.0,18492.0,260416.0,12.34,58.0,X-LC,6.86,7.0,0.45,1.89,24.04,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.40
1,25,44.95
2,50,76.71


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.82
MC    30.87
LC    24.30
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.65
X40      23.16
X40N     12.96
XR        9.78
XY25      9.12
AR        9.00
OX40N     7.55
SR        1.00
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.38
H-SC    23.75
X-LC    20.82
M-SC    11.62
X-SC     8.04
H-MC     4.82
L-SC     1.41
M-LC     1.33
M-MC     1.33
H-LC     1.12
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.36,-9.20,44.31
IT,13.17,-15.97,74.29
FINANCE,9.45,-22.68,76.04
MISC,7.29,-29.58,82.59
ELECTRICAL,6.00,-13.58,55.18
PAINTS,5.84,-14.98,32.07
INSURANCE,4.79,-2.82,38.01
PHARMA,4.13,-3.41,35.96
AUTO,2.81,-35.56,82.56


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3255992.0,21
AR,1331811.0,10
XR,1317136.0,13
X40,1035705.0,14
X40N,901545.0,9
OX40N,754186.0,10
XY25,386185.0,6
SR,283378.0,2
MH,78325.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3713285.0,25
X-MC,1470202.0,16
M-SC,1460813.0,15
X-LC,861311.0,11
X-SC,812619.0,8
H-MC,415214.0,3
L-SC,267146.0,3
M-LC,108845.0,1
H-LC,79287.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1302189.0      6
           AR         907683.0      5
M-SC       XY24       835217.0      6
H-SC       XR         794878.0      7
X-MC       X40        511834.0      7
           XY24       414216.0      3
           X40N       357531.0      4
H-SC       OX40N      346832.0      4
X-LC       X40        328514.0      5
M-SC       OX40N      320385.0      5
X-SC       X40N       314333.0      3
           XY24       302929.0      3
H-SC       SR         283378.0      2
X-LC       X40N       229681.0      2
H-MC       AR         217594.0      2
X-LC       XY24       203821.0      2
H-MC       XY24       197620.0      1
X-SC       X40        195357.0      2
X-MC       XY25       186621.0      2
L-SC       XR         180177.0      2
M-SC       XR         177964.0      2
           AR         127247.0      2
M-LC       XR         108845.0      1
X-LC       XY25        99295.0      2
L-SC       OX40N       86969.0      1
H-LC       AR          79287.0      1
H-SC       MH          78325.0      1
M-MC       XY25        62813.0      1
L-MC       XR          55272.0      1
L-LC       XY25        37456.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
